In [46]:
# basics
from datetime import datetime
import time
import os 
import random
# import gresearch_crypto

# plotting
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

# ml shit
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.stats as stats
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
# stat shit
import statsmodels.api as sm
from statsmodels.graphics import tsaplots
# models
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor


data_root = "../data/"
SEED = 2021
REMOVE_LB_TEST_OVERLAPPING_DATA = True

In [2]:
# setting all seeds
def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

fix_all_seeds(SEED)

In [3]:
train = pd.read_csv(data_root+"train.csv")
asset_d = pd.read_csv(data_root+"asset_details.csv")

In [4]:
if REMOVE_LB_TEST_OVERLAPPING_DATA:
    train["datetime"] = pd.to_datetime(train["timestamp"], unit="s")
    train = train[train["datetime"]<"2021-06-13 00:00:00"]


In [106]:
#basic functions

#this one is prob redundant
to_datetime = lambda timestamp: datetime.strftime(datetime.fromtimestamp(timestamp),"%Y-%m-%d")

get_month_timestamp = lambda timestamp: to_datetime(timestamp).month

get_month = lambda datetime: datetime.month

get_month_day = lambda datetime: datetime.strftime("%m-%d")

############# Two new features from the competition tutorial
def log_return(series, periods=1):
    return np.log(series).diff(periods=periods)

def roll(array, shift):
    # this supposebly improves the performance of np.roll
    if not (isinstance(array, np.ndarray)):
        array = np.asarray(array)
    idx = shift%len(array)
    return np.concatenate([array[-idx:], array[:-idx]])


############# from lightGBT tutorial
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

############# realized here needs to use some data for calculation of initial values
def realized(close, N=240):
    rt = list(np.log(C_t / C_t_1) for C_t, C_t_1 in zip(close[1:], close[:-1]))
    rt_mean = sum(rt) / len(rt)
    return np.sqrt(sum((r_i - rt_mean) ** 2 for r_i in rt) * N / (len(rt) - 1))

########## function for calling all feature creating functions
def get_features(df, Lag=True):
    df_feat = df[["Count", "Open","High", "Low", "Close", "Volume","VWAP", "Target"]].copy()
    
    df_feat["Upper_shadow"] = upper_shadow(df_feat)
    df_feat["Lower_shadow"] = lower_shadow(df_feat)

    df_feat["Volume"] = log_return(df_feat["Volume"]) # maybe Volume is fine just like that(?)
    df_feat["Count"] = log_return(df_feat["Count"])
    df_feat["VWAP"] = log_return(df_feat["VWAP"])
    df_feat = df_feat[1:] # compensate
    if Lag:
        for lag in range(1, 6):
            roll_feature = "VWAP"
            df_feat["rolled_"+roll_feature+f"_{lag}"] = \
                roll(df_feat[roll_feature].values, lag)
    
    ########### for now, simple dropna()     ########### 
    ########### later we can use reindexing  ########### 
    df_feat.dropna(inplace=True)
    # df_feat = df_feat.reindex(range(btc.index[0],btc.index[-1]+60, 60), method="pad")

    return df_feat


def Xy_model_asset(train, asset_id, Lag=True):
    df = train[train["Asset_ID"]==asset_id]

    # todo : try different features here 
    #        also, scale the features
    df_proc = get_features(df, Lag)

    df_proc["y"] = df["Target"]
    # 念の為
    df_proc.dropna(how="any", inplace=True)
    X = df_proc.drop("y", axis=1)
    y = df_proc["y"]

    # todo : try different models here
    # model = LGBMRegressor()
    # model.fit(X, y)
    return X, y#, model


def scorer(model, X, y):
    # not sure how this line works, just trying
    y_pred = model.predict([X])[0]
    score = np.corrcoef(y_pred, y)
    return score

my_scorer = make_scorer(scorer)

In [112]:
train_assets = train.groupby("Asset_ID")
# asset_dict = {name:train_assets[group] for name, group in zip(asset_d.Asset_Name, asset_d.Asset_ID)}

# We first create simple LGBM Baseline

In [54]:
model = LGBMRegressor()
X, y = Xy_model_asset(train, 1)
print(type(X), type(y))
scores = cross_val_score(model, X, y, cv = 5)#, scoring=scorer)
print(scores, "\n", np.mean(scores))


<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
0.9355156853655835


,Asset_ID,Weight,Asset_Name
0,2,2.397895,Bitcoin Cash
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
3,5,1.386294,EOS.IO
4,7,2.079442,Ethereum Classic
5,6,5.894403,Ethereum
6,9,2.397895,Litecoin
7,11,1.609438,Monero
8,13,1.791759,TRON
9,12,2.079442,Stellar
